In [428]:
import pandas as pd
# Crie um DataFrame vazio com as colunas desejadas
columns = ['Name', 'Agente', 'Rating(G / A / D)', 'ACS(G / A / D)', 'Kills(G / A / D)', 'Deaths(G / A / D)', 'Assists(G / A / D)', 'Time', 'Time adversário', 'Mapa', 'Resultado Mapa', 'Resultado Serie']
df = pd.DataFrame(columns=columns)

In [429]:
import requests
from bs4 import BeautifulSoup

# Defina o intervalo de IDs das partidas que você deseja acessar
inicio_id = 280324
fim_id = 280349

for id_partida in range(inicio_id, fim_id + 1):
    
    # Verifique se o ID da partida está na lista de IDs a serem ignorados
    if id_partida in (280338, 280339):
        print(f"Ignorando a partida {id_partida} devido à exceção.")
        continue
    
    # Construa a URL da partida
    url = "https://www.vlr.gg/" + str(id_partida)

    # Realize uma solicitação GET para obter o conteúdo da página
    response = requests.get(url)
    

    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Parse o conteúdo da página usando BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Imprima uma mensagem indicando o sucesso do processamento da página
        print(f"Analisando a página da partida {id_partida}")
        
        tabelas = soup.find_all('div', class_='vm-stats-game')
    
        
        for tabela in tabelas:
            modo = 'normal'
            mapa = tabela.find('div', class_='map')
            if mapa:
                mapa = mapa.text.strip().split()
                # print(mapa)
            else:
                # print("Geral")
                modo = 'geral'
                
                
            if modo != 'geral':
            
                rounds_time1 = int(tabela.find('div', class_="team").text.strip().split()[0])
                # print(rounds_time1)
                rounds_time2 = int(tabela.find('div', class_="team mod-right").text.strip().split()[-1])
                # print(rounds_time2)
                partidas_times = soup.find('div', class_="match-header-vs-score")
                partidas_times = partidas_times.find('div', class_='js-spoiler')
                partidas_time1 = int(partidas_times.text.strip().split()[0])
                partidas_time2 = int(partidas_times.text.strip().split()[2])
                    
                # Encontre a tabela de jogadores, a estrutura da tabela pode variar
                player_table = tabela.find_all('table', class_='wf-table-inset mod-overview')
                
                # Verifique se a tabela foi encontrada
                i = 0
                for table in player_table:
                    # Encontre as linhas da tabela
                    rows = table.find_all('tr')
                    
                    time1 = soup.find('div', class_="match-header-link-name mod-1")
                    time1 = time1.find('div', class_='wf-title-med').text.strip().split()
                    time2 = soup.find('div', class_="match-header-link-name mod-2")
                    time2 = time2.find('div', class_='wf-title-med').text.strip().split()
                    
                    if i == 0:
                        time = time1
                        time_adv = time2
                        rounds = rounds_time1 - rounds_time2
                        partidas = partidas_time1 - partidas_time2
                        # print(rounds)
                    
                    else:
                        time = time2
                        time_adv = time1
                        rounds = rounds_time2 - rounds_time1
                        partidas = partidas_time2 - partidas_time1
                        # print(rounds)
                    i+=1
                    
                    time
                    
                    
                    # Itere pelas linhas da tabela
                    for row in rows[1:]:  # Ignorar o cabeçalho da tabela
                        # Encontre o elemento <td> com a classe 'mod-stat mod-vlr-kills'
                        name = row.find('td', class_='mod-player')
                        agente = row.find('td', class_='mod-agents')
                        stats = row.find_all('td', class_='mod-stat')
                        kills_td = row.find('td', class_='mod-stat mod-vlr-kills')
                        death_td = row.find('td', class_='mod-stat mod-vlr-deaths')
                        assist_td = row.find('td', class_='mod-stat mod-vlr-assists')
                        agentes = []

                        # Agora, você pode obter o conteúdo desse elemento
                        if kills_td:
                            name = name.text.strip().split()
                            nome_agente = agente.find_all('img')
                            
                            for nome in nome_agente:
                                agente = nome.get('title')
                                agentes = agentes + [agente]
                            rating = stats[0].text.strip().split()
                            acs = stats[1].text.strip().split()
                            kills_content = kills_td.text.strip().split()
                            death_td = death_td.text.strip().split()
                            death_td = death_td[1:-1]
                            assist_td = assist_td.text.strip().split()
                            
                            if partidas > 0:
                                resultado = 'Vitoria'
                            else:
                                resultado = 'Derrota'
                                
                                
                            if rounds > 0:
                                resultado_mapa = 'Vitoria'
                            else:
                                resultado_mapa = 'Derrota'
                                
      
                            data = {
                                'Name': name[0],
                                'Agente': agentes[0],
                                'Rating(G / A / D)': rating,
                                'ACS(G / A / D)': acs,
                                'Kills(G / A / D)': kills_content,
                                'Deaths(G / A / D)': death_td,
                                'Assists(G / A / D)': assist_td,
                                'Time': time[0],
                                'Time adversário': time_adv[0],
                                'Mapa': mapa[0],
                                'Resultado Mapa': resultado_mapa,
                                'Resultado Serie': resultado
                            }

                            df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
                #             print(df)
                            
                            
    else:
        print(f"Falha ao acessar a página da partida {id_partida}")


Analisando a página da partida 280324
Analisando a página da partida 280325
Analisando a página da partida 280326
Analisando a página da partida 280327
Analisando a página da partida 280328
Analisando a página da partida 280329
Analisando a página da partida 280330
Analisando a página da partida 280331
Analisando a página da partida 280332
Analisando a página da partida 280333
Analisando a página da partida 280334
Analisando a página da partida 280335
Analisando a página da partida 280336
Analisando a página da partida 280337
Ignorando a partida 280338 devido à exceção.
Ignorando a partida 280339 devido à exceção.
Analisando a página da partida 280340
Analisando a página da partida 280341
Analisando a página da partida 280342
Analisando a página da partida 280343
Analisando a página da partida 280344
Analisando a página da partida 280345
Analisando a página da partida 280346
Analisando a página da partida 280347
Analisando a página da partida 280348
Analisando a página da partida 28034

In [430]:
df

,Name,Agente,Rating(G / A / D),ACS(G / A / D),Kills(G / A / D),Deaths(G / A / D),Assists(G / A / D),Time,Time adversário,Mapa,Resultado Mapa,Resultado Serie
0,mwzera,Jett,"[1.95, 1.72, 2.19]","[373, 352, 398]","[34, 16, 18]","[10, 6, 4]","[1, 1, 0]",FURIA,timberknick,Breeze,Vitoria,Vitoria
1,kon4n,Sova,"[1.22, 1.55, 0.85]","[226, 315, 130]","[18, 14, 4]","[12, 6, 6]","[4, 2, 2]",FURIA,timberknick,Breeze,Vitoria,Vitoria
2,Khalil,Cypher,"[1.01, 1.01, 1.02]","[180, 189, 171]","[15, 9, 6]","[14, 8, 6]","[5, 1, 4]",FURIA,timberknick,Breeze,Vitoria,Vitoria
3,liazzi,Viper,"[0.92, 0.69, 1.17]","[150, 130, 172]","[13, 6, 7]","[13, 8, 5]","[5, 2, 3]",FURIA,timberknick,Breeze,Vitoria,Vitoria
4,havoc,Kayo,"[0.70, 0.58, 0.83]","[142, 133, 152]","[11, 5, 6]","[17, 11, 6]","[12, 8, 4]",FURIA,timberknick,Breeze,Vitoria,Vitoria
...,...,...,...,...,...,...,...,...,...,...,...,...
585,axeddy,Raze,"[1.58, 1.40, 1.64]","[375, 250, 418]","[21, 4, 17]","[12, 2, 10]","[4, 0, 4]",RED,Stellae,Split,Vitoria,Vitoria
586,Leozin,Jett,"[1.45, 2.86, 0.98]","[309, 552, 229]","[18, 9, 9]","[10, 1, 9]","[3, 0, 3]",RED,Stellae,Split,Vitoria,Vitoria
587,cortezia,Astra,"[1.40, 1.16, 1.48]","[189, 142, 205]","[12, 2, 10]","[8, 2, 6]","[11, 5, 6]",RED,Stellae,Split,Vitoria,Vitoria
588,gaabxx,Skye,"[1.32, 1.92, 1.12]","[196, 223, 188]","[12, 3, 9]","[7, 1, 6]","[10, 5, 5]",RED,Stellae,Split,Vitoria,Vitoria


In [431]:
#criar outro dataframe com apenas os primeiros dados de colunas que tem mais de um
#pois alguns dados sao dividios em geral, ataque, defesa
df_geral = df
df_geral['Rating'] = df_geral['Rating(G / A / D)'].apply(lambda x: x[0])
df_geral['ACS'] = df_geral['ACS(G / A / D)'].apply(lambda x: x[0])
df_geral['Kills'] = df_geral['Kills(G / A / D)'].apply(lambda x: x[0])
df_geral['Deaths'] = df_geral['Deaths(G / A / D)'].apply(lambda x: x[0])
df_geral['Assists'] = df_geral['Assists(G / A / D)'].apply(lambda x: x[0])
# drop colunas que não serão mais utilizadas e colocar as novas colunas em seus lugares
df_geral = df_geral.drop(['Rating(G / A / D)', 'ACS(G / A / D)', 'Kills(G / A / D)', 'Deaths(G / A / D)', 'Assists(G / A / D)'], axis=1)
# Selecione as últimas 5 colunas
colunas = ['Name', 'Agente', 'Rating', 'ACS', 'Kills', 'Deaths', 'Assists']

df_geral = df_geral[colunas + [col for col in df_geral.columns if col not in colunas]]

In [432]:
df_geral

,Name,Agente,Rating,ACS,Kills,Deaths,Assists,Time,Time adversário,Mapa,Resultado Mapa,Resultado Serie
0,mwzera,Jett,1.95,373,34,10,1,FURIA,timberknick,Breeze,Vitoria,Vitoria
1,kon4n,Sova,1.22,226,18,12,4,FURIA,timberknick,Breeze,Vitoria,Vitoria
2,Khalil,Cypher,1.01,180,15,14,5,FURIA,timberknick,Breeze,Vitoria,Vitoria
3,liazzi,Viper,0.92,150,13,13,5,FURIA,timberknick,Breeze,Vitoria,Vitoria
4,havoc,Kayo,0.70,142,11,17,12,FURIA,timberknick,Breeze,Vitoria,Vitoria
...,...,...,...,...,...,...,...,...,...,...,...,...
585,axeddy,Raze,1.58,375,21,12,4,RED,Stellae,Split,Vitoria,Vitoria
586,Leozin,Jett,1.45,309,18,10,3,RED,Stellae,Split,Vitoria,Vitoria
587,cortezia,Astra,1.40,189,12,8,11,RED,Stellae,Split,Vitoria,Vitoria
588,gaabxx,Skye,1.32,196,12,7,10,RED,Stellae,Split,Vitoria,Vitoria


In [433]:
df_geral['ACS'] = df_geral['ACS'].astype(float)

In [434]:
df_geral['Rating'] = df_geral['Rating'].astype(float)

In [435]:
df_geral['Rating'].mean()

1.0009322033898305

In [436]:
df_geral['ACS'].mean()

198.6491525423729

In [437]:
# Agrupe os dados por jogador e calcule a média de Rating e ACS
df_avg = df_geral.groupby('Name').agg({'Rating': 'mean', 'ACS': 'mean'}).reset_index()

# Renomeie as colunas para "Nome", "Média Rating" e "Média ACS"
df_avg.columns = ['Nome', 'Média Rating', 'Média ACS']

In [438]:
df_avg.sort_values(by='Média Rating', ascending=False)

,Nome,Média Rating,Média ACS
55,mwzera,1.291429,293.857143
37,gtn,1.200000,218.875000
36,gobera,1.190000,256.428571
23,Zanatsu,1.188000,262.300000
72,skz,1.187143,202.428571
...,...,...,...
0,1buugg,0.706000,161.600000
24,antG,0.640000,160.500000
69,sayuri,0.637500,150.500000
52,m4ndzin,0.605000,163.750000


In [439]:
# Agrupe os dados por jogador e calcule a média de Rating e ACS
df_avg_agente = df_geral.groupby('Agente').agg({'Rating': 'mean', 'ACS': 'mean'}).reset_index()

# Renomeie as colunas para "Nome", "Média Rating" e "Média ACS"
df_avg_agente.columns = ['Agente', 'Média Rating', 'Média ACS']
# resetar o index
df_avg_agente = df_avg_agente.sort_values(by='Média Rating', ascending=False)
df_avg_agente = df_avg_agente.reset_index(drop=True)

In [440]:
df_avg_agente

,Agente,Média Rating,Média ACS
0,Reyna,1.145000,192.000000
1,Chamber,1.101538,239.230769
2,Raze,1.071087,249.086957
3,Breach,1.042963,178.518519
4,Jett,1.021053,225.855263
5,Omen,1.015333,186.983333
6,Kayo,1.004667,196.911111
7,Brimstone,1.003158,191.052632
8,Sova,1.002787,189.000000
9,Sage,0.985000,210.000000


In [447]:
#contar quantas vezes cada mapa aparece
df_mapa = df_geral['Mapa'].value_counts()
df_mapa = df_mapa.reset_index()
#renomear coluna count para 'Quantidade'
df_mapa.columns = ['Mapa', 'Quantidade']
df_mapa['Quantidade'] = (df_mapa['Quantidade']/10).astype(int)

In [450]:
df_mapa

,Mapa,Quantidade
0,Haven,13
1,Ascent,12
2,Breeze,10
3,Bind,10
4,Split,6
5,Lotus,6
6,Sunset,2
